In [ ]:
!pip -q install unsloth accelerate bitsandbytes transformers datasets
!pip -q install trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.1/376.1 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 794.9/794.9 kB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
import os

# Create a folder in the current working directory
os.mkdir('cot_dataset')

In [4]:
# ======================================================================
# 1️⃣ IMPORTS
# ======================================================================
import json
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

from datasets import Dataset
from unsloth import FastLanguageModel
from trl import SFTTrainer
from transformers import TrainingArguments

print("✅ Imports done")

# ======================================================================
# 2️⃣ CONFIGURATION
# ======================================================================
class Config:
    MODEL_NAME = "microsoft/Phi-3.5-mini-instruct"
    OUTPUT_DIR = "./phi-mcq-skill-exp-cot"

    DATASET_PATHS = [
        "/content/cot_dataset/Chain-of-Thought-Cloud.json",
        "/content/cot_dataset/Chain-of-Thought-Deep-Learning.json",
        "/content/cot_dataset/Chain-of-Thought-DSA.json",
        "/content/cot_dataset/Chain-of-Thought-LLM.json",
        "/content/cot_dataset/Chain-of-Thought-Machine-Learning.json",
        "/content/cot_dataset/Chain-of-Thought-Python.json",
        "/content/cot_dataset/Chain-of-Thought-Web.json"
    ]

    MAX_LENGTH = 1536   # Use larger max length for better coverage
    TRAIN_TEST_SPLIT = 0.05

    NUM_EPOCHS = 3
    BATCH_SIZE = 8
    GRAD_ACC = 2

    LR = 3e-4
    WARMUP = 50
    WEIGHT_DECAY = 0.01

    # LoRA parameters
    LORA_R = 32
    LORA_ALPHA = 64
    LORA_DROPOUT = 0.05

config = Config()

# ======================================================================
# 3️⃣ LOAD + PARSE DATASETS
# ======================================================================
def parse_input(input_text):
    skill, exp = "General", "0-2 years"
    for line in input_text.strip().split("\n"):
        if line.startswith("Skills:"):
            skill = line.split(":",1)[1].strip()
        if line.startswith("Experience:"):
            exp = line.split(":",1)[1].strip()
    return skill, exp

def parse_output(output):
    try:
        mcq = output["mcq"][0]
        question = mcq["text"]
        explanation = mcq["explanation"]
        labels = ["A","B","C","D"]

        option_dict = {}
        correct_label = None
        for idx,opt in enumerate(mcq["options"]):
            label = labels[idx]
            option_dict[label] = opt["text"]
            if opt["answerType"]=="CORRECT":
                correct_label = label

        return {"question":question, "options":option_dict, "correct_answer":correct_label, "explanation":explanation}
    except:
        return None

def load_dataset(path):
    with open(path,"r",encoding="utf-8") as f:
        raw = json.load(f)

    parsed = []
    seen_questions = set()
    for item in tqdm(raw, desc=f"Loading {path}"):
        skill, exp = parse_input(item["input"])
        out = parse_output(item["output"])
        if out is None or out["question"] in seen_questions:
            continue
        seen_questions.add(out["question"])
        parsed.append({
            "skill": skill,
            "experience": exp,
            "question": out["question"],
            "A": out["options"]["A"],
            "B": out["options"]["B"],
            "C": out["options"]["C"],
            "D": out["options"]["D"],
            "correct_answer": out["correct_answer"],
            "explanation": out["explanation"]
        })
    return parsed

def load_all(paths):
    all_data = []
    for p in paths:
        all_data.extend(load_dataset(p))
    print(f"✅ Total unique examples: {len(all_data)}")
    return all_data

raw_data = load_all(config.DATASET_PATHS)

# ======================================================================
# 4️⃣ PROMPT BUILDER
# ======================================================================
def build_prompt(ex):
    system_msg = "You are an expert MCQ generator. Generate high-quality questions with one correct answer and detailed explanation."
    user_msg = f"""
Generate exactly ONE multiple-choice question.

Skill: {ex['skill']}
Experience Level: {ex['experience']}

Output strictly in JSON:
{{
  "mcq": [
    {{
      "text": "...",
      "options": {{
        "A": "...",
        "B": "...",
        "C": "...",
        "D": "..."
      }},
      "correct_answer": "A/B/C/D",
      "explanation": "..."
    }}
  ]
}}
"""
    assistant = {
        "mcq":[
            {
                "text": ex["question"],
                "options": {"A": ex["A"], "B": ex["B"], "C": ex["C"], "D": ex["D"]},
                "correct_answer": ex["correct_answer"],
                "explanation": ex["explanation"]
            }
        ]
    }

    return f"<|system|>\n{system_msg}<|end|>\n<|user|>\n{user_msg}<|end|>\n<|assistant|>\n{json.dumps(assistant)}<|end|>"

# ======================================================================
# 5️⃣ TOKENIZATION
# ======================================================================
def prepare_dataset(data, tokenizer):
    records = [{"text": build_prompt(ex)} for ex in data]
    ds = Dataset.from_list(records)

    def tokenize(batch):
        tokenized = tokenizer(
            batch["text"],
            truncation=True,
            max_length=config.MAX_LENGTH,
            padding="max_length"
        )
        labels = tokenized["input_ids"].copy()
        labels = [[(t if t!=tokenizer.pad_token_id else -100) for t in seq] for seq in labels]
        tokenized["labels"] = labels
        return tokenized

    tokenized = ds.map(tokenize, batched=True, remove_columns=["text"])
    return tokenized.train_test_split(test_size=config.TRAIN_TEST_SPLIT)

tokenizer = FastLanguageModel.from_pretrained(config.MODEL_NAME)[1]
dataset = prepare_dataset(raw_data, tokenizer)

# ======================================================================
# 6️⃣ LOAD MODEL + LoRA
# ======================================================================
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=config.MODEL_NAME,
    max_seq_length=config.MAX_LENGTH,
    load_in_4bit=True
)
model = FastLanguageModel.get_peft_model(
    model,
    r=config.LORA_R,
    lora_alpha=config.LORA_ALPHA,
    lora_dropout=config.LORA_DROPOUT,
    target_modules=["q_proj","k_proj","v_proj","o_proj"]
)

# ======================================================================
# 7️⃣ TRAINING ARGUMENTS
# ======================================================================
training_args = TrainingArguments(
    output_dir=config.OUTPUT_DIR,
    per_device_train_batch_size=config.BATCH_SIZE,
    per_device_eval_batch_size=config.BATCH_SIZE,
    gradient_accumulation_steps=config.GRAD_ACC,
    learning_rate=config.LR,
    num_train_epochs=config.NUM_EPOCHS,
    warmup_steps=config.WARMUP,
    weight_decay=config.WEIGHT_DECAY,
    logging_steps=20,
    save_steps=500,
    eval_steps=500,
    save_total_limit=2,
    bf16=True,
    report_to="none"
)

# ======================================================================
# 8️⃣ TRAINING
# ======================================================================
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    dataset_text_field=None,
    max_seq_length=config.MAX_LENGTH,
    args=training_args
)

✅ Imports done


Loading /content/cot_dataset/Chain-of-Thought-Cloud.json: 100%|██████████| 2502/2502 [00:00<00:00, 322193.01it/s]
Loading /content/cot_dataset/Chain-of-Thought-Deep-Learning.json: 100%|██████████| 2502/2502 [00:00<00:00, 330774.40it/s]
Loading /content/cot_dataset/Chain-of-Thought-DSA.json: 100%|██████████| 2502/2502 [00:00<00:00, 310799.60it/s]
Loading /content/cot_dataset/Chain-of-Thought-LLM.json: 100%|██████████| 2502/2502 [00:00<00:00, 261934.62it/s]
Loading /content/cot_dataset/Chain-of-Thought-Machine-Learning.json: 100%|██████████| 2502/2502 [00:00<00:00, 260284.45it/s]
Loading /content/cot_dataset/Chain-of-Thought-Python.json: 100%|██████████| 2533/2533 [00:00<00:00, 264849.48it/s]
Loading /content/cot_dataset/Chain-of-Thought-Web.json: 100%|██████████| 2085/2085 [00:00<00:00, 250504.84it/s]


✅ Total unique examples: 9435
==((====))==  Unsloth 2025.12.9: Fast Llama patching. Transformers: 4.57.3.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 8.0. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.26G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/140 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Map:   0%|          | 0/9435 [00:00<?, ? examples/s]

==((====))==  Unsloth 2025.12.9: Fast Llama patching. Transformers: 4.57.3.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 8.0. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.12.9 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [5]:
print("🚀 Training started...")
trainer.train()
print("🎉 Training completed!")

The model is already on multiple devices. Skipping the move to device specified in `args`.


🚀 Training started...


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 8,963 | Num Epochs = 3 | Total steps = 1,683
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 2 x 1) = 16
 "-____-"     Trainable parameters = 25,165,824 of 3,846,245,376 (0.65% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
20,2.168100
40,0.812600
60,0.590600
80,0.505100
100,0.475000
120,0.462000
140,0.440100
160,0.406500
180,0.393800
200,0.374900


🎉 Training completed!


In [6]:
# ======================================================================
# 9️⃣ SAVE MODEL
# ======================================================================
trainer.save_model(f"{config.OUTPUT_DIR}/final_model")
tokenizer.save_pretrained(f"{config.OUTPUT_DIR}/final_model")
print(f"✅ Model saved at {config.OUTPUT_DIR}/final_model")


# ======================================================================
# EVALUATE
# ======================================================================

eval_results = trainer.evaluate()
print(f"✅ Eval results: {eval_results}")

Unsloth: Not an error, but LlamaForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


✅ Model saved at ./phi-mcq-skill-exp-cot/final_model


✅ Eval results: {'eval_loss': 0.1881784349679947, 'eval_runtime': 32.5558, 'eval_samples_per_second': 14.498, 'eval_steps_per_second': 1.812, 'epoch': 3.0}


In [7]:
# DOWNLOAD

!zip -r phi-mcq-skill-exp-cot.zip {config.OUTPUT_DIR}
from google.colab import files
files.download(f'phi-mcq-skill-exp-cot.zip')
print("✅ Download started")

  adding: phi-mcq-skill-exp-cot/ (stored 0%)
  adding: phi-mcq-skill-exp-cot/checkpoint-1683/ (stored 0%)
  adding: phi-mcq-skill-exp-cot/checkpoint-1683/adapter_model.safetensors (deflated 7%)
  adding: phi-mcq-skill-exp-cot/checkpoint-1683/special_tokens_map.json (deflated 76%)
  adding: phi-mcq-skill-exp-cot/checkpoint-1683/added_tokens.json (deflated 62%)
  adding: phi-mcq-skill-exp-cot/checkpoint-1683/adapter_config.json (deflated 56%)
  adding: phi-mcq-skill-exp-cot/checkpoint-1683/tokenizer_config.json (deflated 86%)
  adding: phi-mcq-skill-exp-cot/checkpoint-1683/training_args.bin (deflated 53%)
  adding: phi-mcq-skill-exp-cot/checkpoint-1683/scheduler.pt (deflated 62%)
  adding: phi-mcq-skill-exp-cot/checkpoint-1683/optimizer.pt (deflated 9%)
  adding: phi-mcq-skill-exp-cot/checkpoint-1683/chat_template.jinja (deflated 61%)
  adding: phi-mcq-skill-exp-cot/checkpoint-1683/README.md (deflated 65%)
  adding: phi-mcq-skill-exp-cot/checkpoint-1683/trainer_state.json (deflated 76%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Download started


### Inference Code

In [10]:
# IMPORTS
import json
import re
from unsloth import FastLanguageModel
from transformers import TextStreamer
import pprint
import torch

MODEL_PATH = "/content/phi-mcq-skill-exp-cot/final_model"

# Load model + tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_PATH,
    max_seq_length=1536,
    load_in_4bit=True
)

print("✅ Finetuned model loaded!")

# =========================
# CLEAN OUTPUT
# =========================
def clean_output(text):
    text = text.replace("<|assistant|>", "").strip()
    return text

# =========================
# BUILD PROMPT FOR CHAIN-OF-THOUGHT
# =========================
def build_prompt(skill="Python", experience="0-2 years"):
    return f"""
You are an expert MCQ generator.
First, write your step-by-step reasoning in natural language about how you will create the MCQ for:
Skill: {skill}, Experience: {experience}.

Explain:
1. Which concept or topic to test.
2. How to phrase the question clearly.
3. How to create plausible distractors.
4. How to determine the correct answer and why.

After your reasoning, output exactly one multiple-choice question in JSON format:

[Reasoning]
Your step-by-step explanation goes here.

[MCQ JSON]
{{
  "mcq": [
    {{
      "text": "...",
      "options": {{"A": "...","B": "...","C": "...","D": "..." }},
      "correct_answer": "A/B/C/D",
      "explanation": "..."
    }}
  ]
}}
"""

# =========================
# GENERATE SINGLE MCQ WITH REASONING
# =========================
def generate_mcq_with_reasoning(skill="Python", experience="0-2 years"):
    prompt = build_prompt(skill, experience)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    # Dynamically set max_new_tokens to avoid truncation
    prompt_len = inputs["input_ids"].shape[1]
    max_tokens = prompt_len + 512  # 512 tokens for reasoning + MCQ
    max_tokens = min(max_tokens, 2048)  # avoid excessive memory usage

    # Generate output with sampling
    output = model.generate(
        **inputs,
        max_new_tokens=max_tokens,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.1,
        pad_token_id=tokenizer.eos_token_id,
    )

    # Decode the generated text
    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    decoded = clean_output(decoded)

    return decoded

# =========================
# GENERATE N MCQs
# =========================
def generate_n_mcqs(n=5, skill="Python", experience="0-2 years"):
    results = []
    for i in range(n):
        print(f"\n=== Generating MCQ {i+1}/{n} ===")
        mcq_text = generate_mcq_with_reasoning(skill, experience)
        print("\n--- Model Reasoning + MCQ ---\n", mcq_text)
        results.append(mcq_text)
    return results

==((====))==  Unsloth 2025.12.9: Fast Llama patching. Transformers: 4.57.3.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 8.0. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
✅ Finetuned model loaded!


In [11]:
# =========================
# EXAMPLE USAGE
# =========================
# Adjust the skill and experience as needed
mcqs = generate_n_mcqs(n=5, skill="Python/LLM", experience="5 years")


=== Generating MCQ 1/5 ===

--- Model Reasoning + MCQ ---
 You are an expert MCQ generator. 
First, write your step-by-step reasoning in natural language about how you will create the MCQ for:
Skill: Python/LLM, Experience: 5 years.

Explain:
1. Which concept or topic to test.
2. How to phrase the question clearly.
3. How to create plausible distractors.
4. How to determine the correct answer and why.

After your reasoning, output exactly one multiple-choice question in JSON format:

[Reasoning]
Your step-by-step explanation goes here.

[MCQ JSON]
{
  "mcq": [
    {
      "text": "...",
      "options": {"A": "...","B": "...","C": "...","D": "..." },
      "correct_answer": "A/B/C/D",
      "explanation": "..."
    }
  ]
}
 [Reasoning]
To test understanding of decorators (a key feature used extensively throughout a typical LLM stack), I'll choose a scenario where users need to apply such knowledge practically. The goal is to assess whether they can identify when and how to use various

In [12]:
# =========================
# EXAMPLE USAGE
# =========================
# Adjust the skill and experience as needed
mcqs = generate_n_mcqs(n=5, skill="DL/Cloud", experience="15 years")


=== Generating MCQ 1/5 ===

--- Model Reasoning + MCQ ---
 You are an expert MCQ generator. 
First, write your step-by-step reasoning in natural language about how you will create the MCQ for:
Skill: DL/Cloud, Experience: 15 years.

Explain:
1. Which concept or topic to test.
2. How to phrase the question clearly.
3. How to create plausible distractors.
4. How to determine the correct answer and why.

After your reasoning, output exactly one multiple-choice question in JSON format:

[Reasoning]
Your step-by-step explanation goes here.

[MCQ JSON]
{
  "mcq": [
    {
      "text": "...",
      "options": {"A": "...","B": "...","C": "...","D": "..." },
      "correct_answer": "A/B/C/D",
      "explanation": "..."
    }
  ]
}
 [Reasoning]
To test deep learning and cloud skills at a high level, we should focus on architectural decisions rather than implementation details. The question should assess understanding of trade-offs between different system designs.

Text: When designing a distri

### DPO

In [ ]:
# ======================================================================
# 1️⃣ IMPORTS
# ======================================================================
import json
import torch
import warnings
warnings.filterwarnings("ignore")

from datasets import load_dataset, concatenate_datasets
from unsloth import FastLanguageModel
from trl import DPOTrainer, DPOConfig
from transformers import AutoTokenizer

from deepeval import evaluate
from deepeval.test_case import LLMTestCase
from deepeval.metrics import (
    AnswerRelevancyMetric,
    FaithfulnessMetric,
    BiasMetric,
)

print("✅ Imports Ready")

# ======================================================================
# 2️⃣ CONFIG
# ======================================================================
BASE_MODEL = "/content/phi-mcq-skill-exp-cot/final_model"
OUTPUT_DIR = "/content/phi-mcq-skill-exp-cot-dpo"

DPO_DATASET = [
    "/content/dpo_dataset/0-15+_Cloud_Services_DPO.json",
    "/content/dpo_dataset/0-15+_DSA_DPO.json",
    "/content/dpo_dataset/0-15+_Deep_Learning_DPO.json",
    "/content/dpo_dataset/0-15+_LLM_DPO.json",
    "/content/dpo_dataset/0-15+_ML_DPO.json",
    "/content/dpo_dataset/0-15+_Python_DPO.json",
    "/content/dpo_dataset/0-15+_Web_DPO.json",
]

SAMPLES_PER_SKILL = 300
SEED = 42

# ======================================================================
# 3️⃣ HELPERS
# ======================================================================
def serialize_output(output_obj):
    return json.dumps(output_obj, ensure_ascii=False)

def build_prompt(chosen_obj):
    instruction = chosen_obj.get("instruction", "").strip()
    input_text = chosen_obj.get("input", "").strip()
    return f"{instruction}\n\n{input_text}".strip()

# ======================================================================
# 4️⃣ LOAD & PREPARE DATASET
# ======================================================================
all_datasets = []

for file_path in DPO_DATASET:
    print(f"📂 Loading {file_path}")
    ds = load_dataset("json", data_files=file_path, split="train")

    if len(ds) < SAMPLES_PER_SKILL:
        raise ValueError(f"{file_path} has only {len(ds)} samples")

    ds = ds.shuffle(seed=SEED).select(range(SAMPLES_PER_SKILL))

    def transform(example):
        return {
            "prompt": build_prompt(example["chosen"]),
            "chosen": serialize_output(example["chosen"]["output"]),
            "rejected": serialize_output(example["rejected"]["output"]),
        }

    ds = ds.map(transform, remove_columns=ds.column_names)
    all_datasets.append(ds)

dpo_dataset = concatenate_datasets(all_datasets)
print(f"🔥 Total DPO samples: {len(dpo_dataset)}")

# ======================================================================
# 5️⃣ TRAIN / EVAL SPLIT
# ======================================================================
dpo_dataset = dpo_dataset.shuffle(seed=SEED)
split = dpo_dataset.train_test_split(test_size=0.05, seed=SEED)

train_dataset = split["train"]
eval_dataset = split["test"]

print(f"📊 Train: {len(train_dataset)} | Eval: {len(eval_dataset)}")

# ======================================================================
# 6️⃣ LOAD MODEL & TOKENIZER
# ======================================================================
model, tokenizer = FastLanguageModel.from_pretrained(
    BASE_MODEL,
    max_seq_length=1536,
    load_in_4bit=True,
    dtype=None,
    trust_remote_code=True,
)

tokenizer.padding_side = "right"
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id

model = FastLanguageModel.for_training(model)

# ======================================================================
# 7️⃣ LOAD REFERENCE MODEL (FROZEN)
# ======================================================================
ref_model, _ = FastLanguageModel.from_pretrained(
    BASE_MODEL,
    max_seq_length=1536,
    load_in_4bit=True,
    dtype=None,
    trust_remote_code=True,
)

ref_model = FastLanguageModel.for_training(ref_model)
ref_model.eval()
for p in ref_model.parameters():
    p.requires_grad = False

print("📌 Model & Reference Model Loaded")

# ======================================================================
# 8️⃣ DPO CONFIG
# ======================================================================
dpo_config = DPOConfig(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=1,
    num_train_epochs=3,
    learning_rate=1e-6,
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    warmup_steps=200,
    weight_decay=0.01,
    save_steps=500,
    logging_steps=20,
    save_total_limit=2,
    bf16=torch.cuda.is_bf16_supported(),
    fp16=not torch.cuda.is_bf16_supported(),
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": False},
    max_grad_norm=0.1,
    remove_unused_columns=False,
    report_to="none",
    beta=0.05,
    loss_type="ipo",
    max_length=1536,
    max_prompt_length=512,
)

# ======================================================================
# 9️⃣ DPO TRAINER
# ======================================================================
trainer = DPOTrainer(
    model=model,
    ref_model=ref_model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    args=dpo_config,
)

In [ ]:
# Train
trainer.train()
print("✅ DPO Training Complete")

In [ ]:
trainer.save_model(f"{OUTPUT_DIR}/final_model")
tokenizer.save_pretrained(f"{OUTPUT_DIR}/final_model")

print(f"✅ Model saved at {OUTPUT_DIR}/final_model")

In [ ]:
# DOWNLOAD

!zip -r phi-mcq-skill-exp-cot-dpo.zip {OUTPUT_DIR}
from google.colab import files
files.download(f'phi-mcq-skill-exp-cot-dpo.zip')
print("✅ Download started")

In [ ]:
# IMPORTS
import json
import re
from unsloth import FastLanguageModel
from transformers import TextStreamer
import pprint

MODEL_PATH = "/content/phi-mcq-skill-exp-cot-dpo/final_model"

# Load model + tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_PATH,
    max_seq_length=1536,
    load_in_4bit=True
)

print("✅ Finetuned model loaded!")

# PROMPT BUILDER
def build_prompt(skill, experience):
    system_msg = "You are an expert MCQ generator. Create high-quality multiple-choice questions with one correct answer and detailed explanation."
    user_msg = f"""
Generate exactly ONE multiple-choice question.

Skill: {skill}
Experience Level: {experience}

Output strictly in JSON:
{{
  "mcq": [
    {{
      "text": "...",
      "options": {{
        "A": "...",
        "B": "...",
        "C": "...",
        "D": "..."
      }},
      "correct_answer": "A/B/C/D",
      "explanation": "..."
    }}
  ]
}}
"""
    return f"<|system|>\n{system_msg}<|end|>\n<|user|>\n{user_msg}<|end|>\n<|assistant|>\n"

# CLEAN OUTPUT
def clean_output(text):
    # Remove all Phi special tokens like <|end|> etc.
    text = re.sub(r"<\|.*?\|>", "", text)
    return text.strip()


# GENERATE SINGLE MCQ
def generate_mcq(skill="Python", experience="0-2 years"):
    prompt = build_prompt(skill, experience)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    eos_id = tokenizer.convert_tokens_to_ids("<|end|>")

    output = model.generate(
        **inputs,
        max_new_tokens=400,
        temperature=0.4,
        top_p=0.9,
        do_sample=True,
        repetition_penalty=1.1,
        eos_token_id=eos_id
    )

    decoded = tokenizer.decode(output[0])
    if "<|assistant|>" in decoded:
        decoded = decoded.split("<|assistant|>")[-1]

    return clean_output(decoded)

# GENERATE N UNIQUE MCQs
def generate_n_mcqs(n=10, skill="Python", experience="0-2 years"):
    results = []
    seen_questions = set()

    for i in range(n):
        print(f"\n=== Generating MCQ {i+1}/{n} ===\n")
        while True:
            mcq = generate_mcq(skill, experience)
            # Try to extract the question text to avoid duplicates
            try:
                mcq_json = json.loads(mcq)
                question_text = mcq_json["mcq"][0]["text"]
            except:
                continue  # Retry if JSON is invalid

            if question_text not in seen_questions:
                seen_questions.add(question_text)
                results.append(mcq)
                pprint.pprint(mcq)
                break  # move to next MCQ

    return results

In [ ]:
# USAGE
# Example: Generate 10 MCQs for Python, 0-2 years experience
mcqs = generate_n_mcqs(n=10, skill="Data Analysis", experience="2-5 years")